<a href="https://colab.research.google.com/github/eniofv/prev_acao_deep_learning/blob/master/Previsao_acao_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neurais recorrentes
Este notebook faz uma simuação de previsão de preços de ações, a partir de uma série temporal

Como pré requisito é necessario instalar a biblioteca Darts 
o comando é o:  pip install u8darts

In [ ]:
pip install u8darts 


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [45]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.preprocessing import ScalerWrapper
from darts.models import RNNModel
from darts.backtesting import backtest_forecasting
from darts.metrics import mape
from darts.metrics import r2_score
from darts.metrics import coefficient_of_variation
from darts.metrics import rmse
from darts.metrics import mse


import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

### Parâmetros:

In [59]:
# Número de períodos anteriores que serão considerados, 3 - meses - aprox 66 dias úteis
SEQ_LENGTH = 66
# Número de features no último estado oculto
HIDDEN_SIZE = 25
# Número de passos de saída para prever
OUTPUT_LEN = 1
# Número de camadas RNN empilhadas
NUM_LAYERS = 1
# 
BATCH_SIZE = 16
# 
EPOCAS = 400
# Modelo da rede neural - Pode se 'LSTM, 'RNN' ou 'GRU'
MODELO = 'LSTM'
# Diretório ao importar arquivos no colab o diretório default é o content
DIRETORIO = '/content/'
# Nome do arquivo de entrada
NOME_ARQUIVO = ''BBDC4_2Ano.xlsx'
#
DATA_FIM_TREINO = '20190401'
#
ACAO = 'BBDC4'

Será utilizada uma rede neural LSTM, para utlilizacão de outra rede é só substituir  ´LSTM' por 'RNN' ou ´GRU'

In [61]:
modelo_previsao_acao = RNNModel(
    model=MODELO,   # Para mudar o modelo é só substituir LSTM, RNN, ou GRU
    output_length=OUTPUT_LEN,
    hidden_size=HIDDEN_SIZE,
    n_rnn_layers=NUM_LAYERS,
    input_length=SEQ_LENGTH,
    batch_size=BATCH_SIZE,
    n_epochs=EPOCAS,
    optimizer_kwargs={'lr': 1e-3}, 
    model_name=ACAO+'_'+MODELO, log_tensorboard=True
)

Para backtest será utilizada uma rede RNN - o número de épocas é menor devido ao tempo de execução

In [ ]:
modelo_previsao_acao_backtest = RNNModel(
    model='RNN',
    output_length=OUTPUT_LEN,
    hidden_size=HIDDEN_SIZE,
    n_rnn_layers=NUM_LAYERS,
    input_length=SEQ_LENGTH,
    batch_size=32,
    n_epochs=200,
    optimizer_kwargs={'lr': 1e-3},
    model_name=ACAO+'_'+MODELO+'_BACK'
)

Leitura dos dados das Ações

In [ ]:
# Leitura dos dados:
Arquivo = DIRETORIO+NOME_ARQUIVO
df = pd.read_excel(Arquivo)
df.describe()



In [ ]:
series = TimeSeries.from_dataframe(df, 'Data', 'Adj Close')
#
# Separa os dados de treino e validação:
treino, validacao = series.split_after(pd.Timestamp(DATA_FIM_TREINO))
treino.plot(label = 'Treino')
validacao.plot(label = 'Validação')
plt.legend()

In [ ]:
series

In [65]:
# Normaliza a série 
transforma = ScalerWrapper()
treino_transformado = transforma.fit_transform(treino)
validacao_transformado = transforma.transform(validacao)
series_transformada = transforma.transform(series)

In [ ]:
treino

In [ ]:
treino_transformado

# Realiza o treino da rede neural

In [ ]:
modelo_previsao_acao.fit(treino_transformado, validacao_transformado, verbose=True)  

### Imprime os parâmetros utilizados no modelo


In [ ]:
print(  'Parâmetros' )
print(  'Nome ' + str(modelo_previsao_acao.model_name))
print(  'Batch_size ' + str(modelo_previsao_acao.batch_size))
print(  'Optmizer ' + str(modelo_previsao_acao.optimizer))
print(  'Tamanho Entrada ' + str(modelo_previsao_acao.input_length))
#print(  'my_model.input_length ' + str(my_model.)

# **Verifica as previsões no conjunto de validação**

In [68]:
def Avalia_modelo(modelo,dias):
    serie_previsao = modelo.predict(n=dias)
    serie_previsao_normal = transforma.inverse_transform(serie_previsao)
    plt.figure(figsize=(8,5))
    validacao.plot(label='Real')
    serie_previsao_normal.plot(label='Previsão')
    plt.title(' Modelo ' + modelo.model_name + ' MAPE: {:.2f}%'.format(mape(serie_previsao, validacao_transformado))+ '  Dias Previsão: '+str(dias))
    #print(serie_previsao)
    plt.legend();


In [ ]:
dias_avaliacao  = [5 , 10 , 30 ]  
for dias in dias_avaliacao:
    Avalia_modelo(modelo_previsao_acao, dias)

Utiliza o melhor modelo a partir do treino de acordo com a 'validation loss':

In [ ]:
Melhor_modelo = RNNModel.load_from_checkpoint(model_name=ACAO+'_'+MODELO, best=True)
for dias in dias_avaliacao:
    Avalia_modelo(Melhor_modelo, dias)

### Backtesting
Realiza um backtest com um modelo  `RNN` e um horizonte de previsão de 3 meses

In [ ]:
# Realiza o  backtest
backtest_series = backtest_forecasting(series_transformada, modelo_previsao_acao_backtest, pd.Timestamp('20190401'), 
                                       fcast_horizon_n=3, verbose=True)

In [ ]:
backtest_series

In [ ]:
series_transformada[456::]

In [ ]:
plt.figure(figsize=(15,10))
series_transformada.plot(label='Real', lw=2)
backtest_series.plot(label='backtest', lw=2)
plt.legend()
plt.title('Backtest, Série Transformada -  Iniciando em ABR 2019, com um horizonte de 3 meses');
print('MAPE: {:.2f}%'.format(mape(transforma.inverse_transform(series_transformada), 
                                 transforma.inverse_transform(backtest_series))))

In [73]:
serie_normal = transforma.inverse_transform(series_transformada)
serie_backtest_normal = transforma.inverse_transform(backtest_series)


In [ ]:
serie_normal
plt.figure(figsize=(15,10))
serie_normal.plot(label='Real', lw=2)
serie_backtest_normal.plot(label='backtest', lw=2)
plt.legend()
plt.title('Backtest, Série Original Iniciando em ABR 2019, com um horizonte de 3 meses');
print('MAPE: {:.2f}%'.format(mape(transforma.inverse_transform(series_transformada), 
                                 transforma.inverse_transform(backtest_series))))

# **Métricas**

In [ ]:


print('***MÉTRICAS**** - Ação - ' + ACAO)
print('MAPE                    : {:.2f}%'.format(mape(serie_normal, serie_backtest_normal)))
print('Coeficiente de Variação : {:.2f}%'.format(coefficient_of_variation(serie_normal, serie_backtest_normal)))  
print('R2_score                : {:.2f}%'.format(r2_score(serie_normal, serie_backtest_normal)))                             
print('RMSE                    : {:.2f}%'.format(rmse(serie_normal, serie_backtest_normal,True)))
print('MSE                     : {:.2f}%'.format(mse(serie_normal, serie_backtest_normal,True)))